# Install COLMAP

In [1]:
!apt update -qq && apt install -y colmap

import os

236 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcamd2 libccolamd2 libceres2 libcholmod3 libcolamd2 libcxsparse3
  libdouble-conversion3 libevdev2 libfreeimage3 libgflags2.2 libglew2.2
  libgoogle-glog0v5 libgudev-1.0-0 libilmbase25 libinput-bin libinput10
  libjxr0 libmd4c0 libmetis5 libmtdev1 libopenexr25 libpcre2-16-0 libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libraw20
  libspqr2 libsuitesparseconfig5 libwacom-bin libwacom-common libwacom9
  qt5-gtk-platformtheme qttranslations5-l10n
Suggested packages:
  glew-utils qt5-image-formats-plugins qtwayland5
The f

# Set COLMAP to run w/o GUI

In [2]:
os.environ["QT_QPA_PLATFORM"] = "offscreen"

# Prepare directories for extracting images and features

In [3]:
import os

# Dynamically get the current working directory (e.g., /notebooks/, /home/user/, etc.)
base_dir = os.getcwd()

# Or explicitly print it to see what the new base directory is
print("Base directory is:", base_dir)

# Define subdirectories based on base_dir
images_dir = os.path.join(base_dir, 'images')
sparse_dir = os.path.join(base_dir, 'sparse')

# Create directories if they don't exist
os.makedirs(images_dir, exist_ok=True)
os.makedirs(sparse_dir, exist_ok=True)



Base directory is: /notebooks


# Import image dataset zip file

In [5]:
import zipfile

# Look for the first ZIP file in the images directory
zip_files = [f for f in os.listdir(images_dir) if f.endswith('.zip')]

if zip_files:
    zip_path = os.path.join(images_dir, zip_files[0])
    print(f"Found zip file: {zip_path}")
    
    # Extract it in-place into the same images_dir
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(images_dir)
    print("Extraction complete.")
else:
    print("No zip file found in images_dir.")

Found zip file: /notebooks/images/merlion_iphone_nerf.zip
Extraction complete.


In [20]:
# rename the folder containing images to 'iphone', oops
unzipped_images_dir = os.path.join(images_dir, 'iphone')

# Extract features from each image using COLMAP

- Some images had slightly different intrinsic characteristics due to differing zoom levels and auto-post processing in iphone, I think. As a result, we let COLMAP run its own estimation of camera intrinsics for each photo.

In [30]:
!colmap feature_extractor \
    --database_path {base_dir}/database.db \
    --image_path {unzipped_images_dir} \
    --ImageReader.camera_model PINHOLE \
    --SiftExtraction.use_gpu 0 \
    --log_to_stderr 1



Feature extraction

Processed file [1/127]
  Name:            IMG_0001.JPG
  SKIP: Features for image already extracted.
Processed file [2/127]
  Name:            IMG_0002.JPG
  SKIP: Features for image already extracted.
Processed file [3/127]
  Name:            IMG_0003.JPG
  SKIP: Features for image already extracted.
Processed file [4/127]
  Name:            IMG_0004.JPG
  SKIP: Features for image already extracted.
Processed file [5/127]
  Name:            IMG_0005.JPG
  SKIP: Features for image already extracted.
Processed file [6/127]
  Name:            IMG_0006.JPG
  SKIP: Features for image already extracted.
Processed file [7/127]
  Name:            IMG_0007.JPG
  SKIP: Features for image already extracted.
Processed file [8/127]
  Name:            IMG_0008.JPG
  SKIP: Features for image already extracted.
Processed file [9/127]
  Name:            IMG_0009.JPG
  SKIP: Features for image already extracted.
Processed file [10/127]
  Name:            IMG_0010.JPG
  SKIP: Featur

# Match features across all images

In [31]:
!colmap exhaustive_matcher \
    --database_path {base_dir}/database.db \
    --SiftMatching.use_gpu 0


Exhaustive feature matching

Matching block [1/3, 1/3] in 260.656s
Matching block [1/3, 2/3] in 292.846s
Matching block [1/3, 3/3] in 77.440s
Matching block [2/3, 1/3] in 279.834s
Matching block [2/3, 2/3] in 250.341s
Matching block [2/3, 3/3] in 72.375s
Matching block [3/3, 1/3] in 230.491s
Matching block [3/3, 2/3] in 200.635s
Matching block [3/3, 3/3] in 65.942s
Elapsed time: 28.847 [minutes]


In [32]:
mapper_dir = os.path.join(sparse_dir, '0')
os.makedirs(mapper_dir, exist_ok=True)

!colmap mapper \
    --database_path {base_dir}/database.db \
    --image_path {unzipped_images_dir} \
    --output_path {mapper_dir}


Loading database

Loading cameras... 12 in 0.002s
Loading matches... 3333 in 0.047s
Loading images... 127 in 0.192s (connected 127)
Building correspondence graph... in 0.624s (ignored 0)

Elapsed time: 0.015 [minutes]


Finding good initial image pair


Initializing with image pair #97 and #71


Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.179799e+02    0.00e+00    9.11e+04   0.00e+00   0.00e+00  1.00e+04        0    1.46e-03    1.01e-02
   1  5.723417e+01    1.61e+02    5.17e+04   8.21e+01   9.98e-01  3.00e+04        1    4.41e-03    1.45e-02
   2  5.276109e+01    4.47e+00    3.02e+04   1.18e+02   9.88e-01  9.00e+04        1    2.40e-04    1.48e-02
   3  5.544095e+01   -2.68e+00    3.02e+04   2.09e+02  -9.81e-01  4.50e+04        1    1.23e-04    1.49e-02
   4  5.149370e+01    1.27e+00    1.24e+04   1.31e+02   6.45e-01  4.61e+04        1    2.23e-04    1.52e-02
   5  5.017750e+01    1.32e+

In [33]:
model_dir = os.path.join(mapper_dir, "0")

!colmap model_converter \
    --input_path {model_dir} \
    --output_path {model_dir} \
    --output_type TXT

print("COLMAP pipeline complete. Sparse model is ready.")

COLMAP pipeline complete. Sparse model is ready.


In [35]:
import sqlite3

db_path = os.path.join(base_dir, "database.db")
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 1. Count number of images in the database
cursor.execute("SELECT COUNT(*) FROM images")
image_count = cursor.fetchone()[0]
print(f"Number of images with extracted features: {image_count}")

# 2. Optional: Check how many pairwise matches exist
cursor.execute("SELECT COUNT(*) FROM matches")
match_count = cursor.fetchone()[0]
print(f"Number of image pairs with matching features: {match_count}")

conn.close()

upzipped_images_dir = os.path.join(images_dir, 'iphone')
image_files = [f for f in os.listdir(unzipped_images_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
print(f"Found {len(image_files)} image files.")


Number of images with extracted features: 127
Number of image pairs with matching features: 8001
Found 127 image files.
